In [8]:
import random
import numpy as np
import networkx as nx 
from dijkstar import Graph, find_path
from ipywidgets import IntProgress 
from IPython.display import display
import matplotlib.pyplot as plt

In [9]:
def get_complete_path(path, all_paths):
  complete_path = []
  for i in range(len(path) - 1):
    complete_path.extend(all_paths[(path[i], path[i+1])][:-1])
    if i == len(path) - 2:
      complete_path.append(all_paths[(path[i], path[i+1])][-1])
  return complete_path

def is_path_in_interval(path, complete_graph, intervals, interval_vertex):
  for index in range(len(interval_vertex)):
    weight = 0
    for i in range(np.where(np.array(path) == interval_vertex[index])[0][0]):
      weight += complete_graph[path[i]][path[i+1]]
    if weight < intervals[index][0] or weight > intervals[index][1]:
      return False
  return True

In [10]:
def get_path_weight(path, complete_graph):
  weight = 0
  for i in range(len(path) - 1):
    weight += complete_graph[path[i]][path[i+1]]
  return weight

def generate_population(nb_people, complete_graph, interval, interval_vertex, start_vertex):
  population = []
  bar = IntProgress(min=0, max=nb_people, layout={"width" : "100%"})
  display(bar)
  while(len(population) < nb_people):
    path = generate_random_path(len(complete_graph), start_vertex)
    if is_path_in_interval(path, complete_graph, interval, interval_vertex):
      population.append(path)
      bar.value += 1
  bar.close()    
  return population

def get_best_path(population, weights):
  index = np.argmin(weights)
  return population[index], weights[index]


In [11]:
def mutate_path_insert(path, complete_graph, interval, interval_vertex):
  is_path_correct = False
  while not is_path_correct:
    start = path[0]
    path = np.delete(path, np.where(path == start))
    mutation = random.choice(path)
    path = np.delete(path, np.where(path == mutation))
    new_index = random.randint(0, len(path) - 1)
    path = np.insert(path, new_index, mutation)
    path = np.append(path, start)
    path = np.insert(path, 0, start)
    is_path_correct = is_path_in_interval(path, complete_graph, interval, interval_vertex)
  return path

def generate_children(parentA, parentB, complete_graph, interval, interval_vertex):
  start = parentA[0]
  parentA = np.delete(parentA, np.where(np.array(parentA) == start))
  parentB = np.delete(parentB, np.where(np.array(parentB) == start))
  is_path_correct = False
  while not is_path_correct:
    cut_start = random.randint(0, len(parentA) - 2)
    cut_end = random.randint(cut_start + 1, len(parentA) - 1)
    child = parentA[cut_start:cut_end]
    for i in child:
      remaining_parentB = np.delete(parentB, np.where(parentB == i))
    child = np.insert(child, len(child), remaining_parentB)
    child = np.append(child, start)
    child = np.insert(child, 0, start)
    is_path_correct = is_path_in_interval(child, complete_graph, interval, interval_vertex)
  return child

def generate_random_path(nb_vertex, start_vertex):
  path = np.arange(nb_vertex)
  path = np.delete(path, np.where(path == start_vertex))
  np.random.shuffle(path)
  path = np.append(path, start_vertex)
  path = np.insert(path, 0, start_vertex)
  return path

def get_best_population(population):
  return np.argpartition(population, -int(len(population)/4))[:int(len(population)/4)]

def generate_decency(population_number, best_people, complete_graph, interval, interval_vertex_index):
  children = []
  for _ in range(population_number):
    P1 = random.choice(best_people)
    # child = generate_children(P1, P2, complete_graph, interval, interval_vertex_index)
    child = mutate_path_insert(P1, complete_graph, interval, interval_vertex_index)
    children.append(child)
  return children

def evalute_population_weight(population, complete_graph):
  weights = []
  for path in population:
    weight = get_path_weight(path, complete_graph)
    weights.append(weight)
  return weights

def evolve(population, complete_graph, interval, interval_vertex_index):
  best_weight = np.inf
  best_path = []
  all_best_weights = np.array([])
  bar = IntProgress(min=0, max=generation_number, layout={"width" : "100%"})
  display(bar)
  for i in range(generation_number):
    bar.value += 1
    weights = evalute_population_weight(population, complete_graph)
    best_pop_path, best_pop_weight = get_best_path(population, weights)
    all_best_weights = np.append(all_best_weights,best_pop_weight)
    if(best_pop_weight < best_weight):
      best_weight = best_pop_weight
      best_path = best_pop_path
    best_people = np.array(population)[get_best_population(weights)]
    population = generate_decency(population_number, best_people, complete_graph, interval, interval_vertex_index)
  bar.close()
  return best_path, best_weight, all_best_weights

def plot_graph(path, graph, all_vertex, all_verticies, start_vertex):
  for i in range(len(graph) - 1):
    for j in range(i + 1, len(graph)):
      if graph[i, j] != np.inf:
        x = [all_vertex[i][0], all_vertex[j][0]]
        y = [all_vertex[i][1], all_vertex[j][1]]
        plt.plot(x, y, color = "gray")
  for i in range(len(path) - 1):
    x = [all_vertex[path[i]][0], all_vertex[path[i+1]][0]]
    y = [all_vertex[path[i]][1], all_vertex[path[i+1]][1]]
    plt.plot(x, y, color = "red")
  for i in range(len(all_vertex)):
    if(i == start_vertex):
      plt.scatter(all_vertex[i][0], all_vertex[i][1], color = "blue")
    elif(i in all_verticies):
      plt.scatter(all_vertex[i][0], all_vertex[i][1], color = "green")
    else: 
      plt.scatter(all_vertex[i][0], all_vertex[i][1], color = "gray")
    plt.text(all_vertex[i][0] - 10, all_vertex[i][1] + 20, str(i))
  plt.show()

Notre problème est le suivant, nous devons determiner le chemin le plus court entre plusieurs villes pour qu'un camion de livraison puisse parcourir toute ces villes en le moins de temps possible et ainsi livrer tous les colis.

Nous avons à notre disposition une liste de villes et parmis ces villes nous devons en séléctionner quelques unes et determiner le cycle de plus court pour passer par toute ces villes et revenir au point de depart.
<br>
Nous avons aussi décider d'ajouter une contrainte qui est que pour certaines villes, le camion n'aura le droit de livrer des colis que dans une fenêtre horaires bien précise.

Ici, pour modeliser notre problème, nous allons devoir modéliser les differente elements de la façon suivante :

- Le graph de toutes les villes : une matrice d'adjacence
- Le graph complet de toutes les villes choisis : une matrice d'adjacence
- La liste de toute les villes choisis : un tuple des indexs des sommets dans le graph de toutes les villes
- Un sommet de depart : un index dans la liste de toute les villes
- Un sommet de passge avec intervalle : un index dans la liste de toute les villes choisis
- Un intervalle de temps : un tuple 

Les algorithmes génétiques appartiennent à la famille des algorithmes évolutionnistes.
<br>
Leur but est d'obtenir une solution approchée à un problème d'optimisation, lorsqu'il n'existe pas de méthode exacte (ou que la solution est inconnue) pour le résoudre en un temps raisonnable.
<br>
Les algorithmes génétiques utilisent la notion de sélection naturelle et l'appliquent à une population de solutions potentielles au problème donné.

Le principe de cet algorithme est très simple, on va chercher à mimer les comportements que l'on peut retrouver dans la nature (selection naturelle).
Cela va se decomposer en deux étapes :

0. création de la population

1. evaluation de la population
2. selection des individues pour generer la prochaine population
3. reproduction de la population en effectuer des croisements et des mutations

4. répétition des étapes 1 à 3 pour un nombre de generation donné

Les algorithmes génétiques reprennent la théorie de Darwin : sélection naturelle de variations individuelles : les individus les plus adaptés  tendent à survivre plus longtemps et à se reproduire plus aisément que les autres.

Amélioration de la population très rapide au début (recherche globale) puis de plus en plus lente à mesure que le temps passe (recherche locale).

Le temps de calcul théorique des algorithmes génétiques croît en ${n\ln(n)}$ ${n}$ étant le nombre de variables. Ce temps de calcul est donc linéarisé par le nombre de variables.

Nous avons donc décider d'appliquer l'algorithme génétique sur notre problème.

Nous avons 2 type de mutation réaliser :

- La mutation par insertion qui consiste à prendre un sommet un chemin et de la placer à un autre endroit dans ce meme chemin
- La mutation par échange qui consiste à prendre un sommet un chemin et de le remplacer par un autre sommet dans ce meme chemin

Ensuite pour le croisement, nous avons effectuer le croisement parcours croisé :
<br>
On a séléctionné deux individus aléatoirement, on les croise en prenant la moitie de leur chemin et on les fusionne en un nouveau chemin en faisant attention à ne pas créer de doublons.

In [12]:
import pickle
with open("soutenance.pickle", "rb") as file:
  data = pickle.load(file)

graph = data["graph"]
complete_graph = data["complete_graph"]
all_paths = data["all_paths"]
vertices_list = data["vertices_list"]
start_vertex = data["start_vertex"]
start_vertex_index = data["start_vertex_index"]
interval_vertex = data["interval_vertex"]
interval_vertex_index = data["interval_vertex_index"]
interval = data["interval"]

population_number = 100
generation_number = 100

population = generate_population(population_number, complete_graph, interval, interval_vertex_index, start_vertex_index)
best_path, best_weight, all_best_weights = evolve(population, complete_graph, interval, interval_vertex_index)

complete_path = get_complete_path(best_path, all_paths)

data = {
  "complete_path": complete_path,
  "weight": best_weight,
}

with open('soutenance_path.pickle', 'wb') as file:
    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

IntProgress(value=0, layout=Layout(width='100%'))

IntProgress(value=0, layout=Layout(width='100%'))

In [13]:
import pickle
with open("soutenance_interval.pickle", "rb") as file:
  data = pickle.load(file)

graph = data["graph"]
complete_graph = data["complete_graph"]
all_paths = data["all_paths"]
vertices_list = data["vertices_list"]
start_vertex = data["start_vertex"]
start_vertex_index = data["start_vertex_index"]
interval_vertex = data["interval_vertex"]
interval_vertex_index = data["interval_vertex_index"]
interval = data["interval"]

population_number = 100
generation_number = 100

population = generate_population(population_number, complete_graph, interval, interval_vertex_index, start_vertex_index)
best_path, best_weight, all_best_weights = evolve(population, complete_graph, interval, interval_vertex_index)

complete_path = get_complete_path(best_path, all_paths)

data = {
  "complete_path": complete_path,
  "weight": best_weight,
}

with open('soutenance_interval_path.pickle', 'wb') as file:
    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

IntProgress(value=0, layout=Layout(width='100%'))

IntProgress(value=0, layout=Layout(width='100%'))

In [ ]:
# import matplotlib.pyplot as plt

# res = plt.plot(range(len(all_best_weights)), all_best_weights)
# plt.xlabel("generation", fontsize=16)
# plt.ylabel("weight", fontsize=16)